In [8]:
from hestonpy.models.heston import Heston
from hestonpy.portfolio.Portfolio import Portfolio
from hestonpy.portfolio.strategies import naive_strategy, optimal_allocate_strategy, run_strategies

import matplotlib.pyplot as plt 
import numpy as np

### Initialisation of the model

In [9]:
spot = 100
r = 0.05

params = {
    "vol_initial": 0.06,
    "kappa": 1.25,
    "theta": 0.06,
    "drift_emm": 0.00,
    "sigma": 0.6,
    "rho": -0.8,
}

time_to_maturity = 1
heston = Heston(spot=spot, r=r, **params)
S, V, _ = heston.simulate(scheme='milstein', nbr_simulations=100, nbr_points=252)

time = np.linspace(start=0, stop=time_to_maturity, num=len(S))
dt = time[1] - time[0]

portfolio = Portfolio(r=r, dt=dt)

### Naive constant allocation strategy


In [10]:
allocation1 = naive_strategy(0.5, len(S))
bank_account, stocks_account = portfolio.back_test(S=S, portfolio0=spot, allocation_strategy=allocation1)
portfolio_value1 = bank_account + stocks_account

allocation2 = naive_strategy(0.7, len(S))
bank_account, stocks_account = portfolio.back_test(S=S, portfolio0=spot, allocation_strategy=allocation2)
portfolio_value2 = bank_account + stocks_account

allocation3 = naive_strategy(1, len(S))
bank_account, stocks_account = portfolio.back_test(S=S, portfolio0=spot, allocation_strategy=allocation3)
portfolio_value3 = bank_account + stocks_account

### Optimal allocation strategy


In [12]:
p = 0.02
optimal_allocation = optimal_allocate_strategy(heston=heston, p=p, time=time)
bank_account, stocks_account = portfolio.back_test(S=S, portfolio0=spot, allocation_strategy=optimal_allocation)
portfolio_value_optimal = bank_account + stocks_account

AttributeError: 'Heston' object has no attribute 'premium_volatility_risk'

### Run strategies


In [ ]:
seeds = np.arange(1, 1000, 1) + np.random.randint(low=0, high=100000)
run_strategies(seeds, portfolio0=spot)

### Plot strategies

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)  

ax1.plot(time, portfolio_value1, label=r'$\pi_{50\%}$', color='blue', linewidth=1)
ax1.plot(time, portfolio_value2, label=r'$\pi_{75\%}$', color='green', linewidth=1)
ax1.plot(time, portfolio_value3, label=r'$\pi_{100\%}$', color='red', linewidth=1)
ax1.plot(time, portfolio_value_optimal, label=r'$\pi^\star$', color='purple', linewidth=1)

ax1.plot(time, S, label='Stock', color='black', linewidth=1)
ax1.set_ylabel('Value [currency unit]')
ax1.set_title('Portfolio Value over Time')
ax1.legend()
ax1.grid(visible=True)

# P&L
PnL1 = np.diff(portfolio_value1)
PnL2 = np.diff(portfolio_value2)
PnL3 = np.diff(portfolio_value3)
PnL_opt = np.diff(portfolio_value_optimal)

print(
    "Total variation of the strategies",
    f"\n PnL1: {round(np.mean(PnL1**2),2)}",
    f"\n PnL2: {round(np.mean(PnL2**2),2)}"
    f"\n PnL3: {round(np.mean(PnL3**2),2)}"    
    f"\n PnL opt: {round(np.mean(PnL_opt**2),2)}"
)

ax2.plot(time[1:], PnL1, label=r'$\pi_{50\%}$', color='blue', linewidth=0.7)
ax2.plot(time[1:], PnL2, label=r'$\pi_{75\%}$', color='green', linewidth=0.7)
ax2.plot(time[1:], PnL3, label=r'$\pi_{100\%}$', color='red', linewidth=0.7)

ax2.plot(time[1:], PnL_opt, label=r'$\pi^\star$', color='purple', linewidth=0.7)


ax2.set_xlabel('Time')
ax2.set_ylabel(r'$PnL(t)$')
ax2.set_title(r'$PnL$')
ax2.legend()
ax2.grid(visible=True)

plt.suptitle('Asset allocation under Heston')
plt.tight_layout()  
plt.show()


## Surface plot
p = np.arange(start=0, stop=1, step=0.01)
p = p[1:-1]
time = np.arange(start=0, stop=1, step=0.01)
p, tau = np.meshgrid(p, heston.T - time)

Z = optimal_allocate_strategy(heston=heston, p=p, time=tau)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(tau, p, Z)

ax.set_xlabel('p')
ax.set_ylabel(r'$\tau$')
ax.set_zlabel(r'$\%$ of portfolio in stock')

plt.show()